# Response streaming using LMI container on SageMaker
In this tutorial, you will use LMI container from DLC to SageMaker and run inference with it. 
We will use the SageMaker Runtime `InvokeEndpointWithResponseStream` API which returns model responses in a stream-like manner.

Note that this API is currently in beta. DO NOT onboard any production use-cases to this API during this beta program.

Please make sure the following permission granted before running the notebook:

- S3 bucket push access
- SageMaker access

## Step 1: Let's bump up SageMaker and import stuff

The wheel installed here is a private preview wheel, you need to add into allowlist to run this function

In [1]:
%pip install sagemaker pip --upgrade  --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.27.144 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.
jupyterlab-server 2.22.1 requires jsonschema>=4.17.3, but you have jsonschema 3.2.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Note the following may error depending on which awscli is installed in your jupyter kernel, 
# but that is ok 

%pip install botocore-*-py3-none-any.whl boto3-*-py3-none-any.whl --force

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Processing ./botocore-1.29.157-py3-none-any.whl
Processing ./boto3-1.26.157-py3-none-any.whl
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached python_dateutil-2.8.2-py2.py3-none-any.whl (247 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 5.2 MB/s eta 0:00:00
  Using cached s3transfer-0.6.1-py3-none-any.whl (79 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.8
    Uninstalling urllib3-1.26.8:
      Successfully uninstalled urllib3-1.26.8
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: jmespath
    Found existing installation: jmespath 1.0.1
    Uninstalling jmespath-1.0.1:
      Successfully uninstalled jmespath-1.0.1
  Attempting uninstall: python-dateutil
    Fo

In [3]:
!aws configure add-model --service-model file://runtime.sagemaker-2017-05-13.normal.json --service-name sagemaker-runtime-demo

In [1]:
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers

boto3_session=boto3.session.Session(region_name="us-west-2")
smr = boto3.client('sagemaker-runtime-demo')
sm = boto3.client('sagemaker')
role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session(boto3_session, sagemaker_client=sm, sagemaker_runtime_client=smr)  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

In [2]:
print(f"Role: {role}")

Role: arn:aws:iam::687912291502:role/webui-notebook-stack-ExecutionRole-62U5FV4LJQS


## Step 2: Start preparing model artifacts
In LMI contianer, we expect some artifacts to help setting up the model
- serving.properties (required): Defines the model server settings
- model.py (optional): A python file to define the core inference logic
- requirements.txt (optional): Any additional pip wheel need to install

In [22]:
%%writefile serving.properties
engine=Python
option.model_id=THUDM/chatglm-6b
option.tensor_parallel_degree=1
option.dtype=fp16
option.enable_streaming=True
gpu.maxWorkers=1
option.predict_timeout=240

Writing serving.properties


In [23]:
%%writefile requirements.txt
transformers==4.27.1
cpm_kernels

Writing requirements.txt


In [24]:
%%writefile model.py
from djl_python import Input, Output
import os
import deepspeed
import torch
from djl_python.streaming_utils import StreamingUtils
import transformers
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainingArguments,
    set_seed,
)

model = None
tokenizer =None

def model_fn(model_dir):
    tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)
    print("=================model_fn_Start=================")
    model_s3_path = os.environ['MODEL_S3_PATH']
    print("=================model s3 path=================="+model_s3_path)
    os.system("sudo find / -name s5cmd")
    os.system("s5cmd sync {0} {1}".format(model_s3_path+"*","/tmp/model/"))
    if os.environ["MODEL_TYPE"] == "ptuning":
        config = AutoConfig.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, pre_seq_len=128)
        model = AutoModel.from_pretrained("THUDM/chatglm-6b", config=config, trust_remote_code=True)
        prefix_state_dict = torch.load(os.path.join("/tmp/model/", "pytorch_model.bin"))
        new_prefix_state_dict = {}
        for k, v in prefix_state_dict.items():
            if k.startswith("transformer.prefix_encoder."):
                new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
        model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)

    elif os.environ["MODEL_TYPE"] == "full turning":
        print("====================load full turning=================")
        config = AutoConfig.from_pretrained("/tmp/model/", trust_remote_code=True, pre_seq_len=128)
        model = AutoModel.from_pretrained("/tmp/model/", trust_remote_code=True)
    else:
        print("====================load normal ======================")
        config = AutoConfig.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)
        model = AutoModel.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

    #model = model.to("cuda")
    model = model.quantize(4)
    model = model.half().cuda()
    model = model.eval()
    print("=================model_fn_End=================")
    return model,tokenizer



def get_model(properties):
    return model_fn(None)

def stream(query, history):
    if query is None or history is None:
        yield {"query": "", "response": "", "history": [], "finished": True}
    size = 0
    response = ""
    for response, history in model.stream_chat(tokenizer, query, history):
        this_response = response[size:]
        history = [list(h) for h in history]
        size = len(response)
        yield {"delta": this_response, "response": response, "finished": False}
    print("Answer - {}".format(response))
    yield {"query": query, "delta": "[EOS]", "response": response, "history": history, "finished": True}




def handle(inputs: Input) -> None:
    global model, tokenizer
    
    if not model:
        model,tokenizer = get_model(None)
    
    if inputs.is_empty():
        # Model server makes an empty call to warmup the model on startup
        return None

    data = inputs.get_as_json()
    outputs = Output()
    outputs.add_property("content-type", "application/jsonlines")
    outputs.add_stream_content(stream(data['query'],data['history']))
    return outputs

Writing model.py


In [25]:
%%sh
mkdir mymodel
mv serving.properties mymodel/
mv model.py mymodel/
mv requirements.txt mymodel/
# remove the following lines if not needed
tar czvf mymodel.tar.gz mymodel/
rm -rf mymodel

mymodel/
mymodel/serving.properties
mymodel/requirements.txt
mymodel/model.py


## Step 3: Start building SageMaker endpoint
In this step, we will build SageMaker endpoint from scratch

### Getting the container image URI

Available framework are:
- djl-deepspeed (0.20.0, 0.21.0)
- djl-fastertransformer (0.21.0)

In [26]:
image_uri = image_uris.retrieve(
    framework="djl-deepspeed",
    region=sess.boto_session.region_name,
    version="0.22.1"
)

### Upload artifact on S3 and create SageMaker model

In [27]:
s3_code_prefix = "large-model-lmi/code"
bucket = sess.default_bucket()  # bucket to house artifacts
code_artifact = sess.upload_data("mymodel.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")
env = {"HUGGINGFACE_HUB_CACHE": "/tmp", "TRANSFORMERS_CACHE": "/tmp",
    #'MODEL_TYPE'                    : 'ptuning',
    #'MODEL_TYPE'                    : 'full turning',
    'MODEL_TYPE'                    : 'normal',
    #'MODEL_S3_PATH'                 : 's3://sagemaker-us-west-2-687912291502/llm/models/chatglm/simple/adgen-chatglm-6b-ft/'
    'MODEL_S3_PATH'                 : 's3://sagemaker-us-west-2-687912291502/llm/models/chatglm/simple/adgen-chatglm-6b-ft/'
}

model = Model(sagemaker_session=sess, image_uri=image_uri, model_data=code_artifact, env=env, role=role)

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-687912291502/large-model-lmi/code/mymodel.tar.gz


### 4.2 Create SageMaker endpoint

You need to specify the instance to use and endpoint names

In [28]:
instance_type = "ml.g5.4xlarge"
endpoint_name = sagemaker.utils.name_from_base("streaming-test")
model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name
)

---------------!

## Step 5: Test and benchmark the inference

In [29]:
import io


class StreamScanner:
    """
    A helper class for parsing the InvokeEndpointWithResponseStream event stream. 
    
    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```
    
    While usually each PayloadPart event from the event stream will contain a byte array 
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```
    
    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'readlines' function. It maintains the position of the last read 
    position to ensure that previous bytes are not exposed again. 
    """
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0

In [34]:
import json

                
body = {"query": "新加坡有什么好玩的？", "history":[],"parameters": {"max_new_tokens":128, "enable_sampling": "true"}}
resp = smr.invoke_endpoint_with_response_stream(EndpointName=endpoint_name, Body=json.dumps(body), ContentType="application/json")
event_stream = resp['Body']
scanner = StreamScanner()
for event in event_stream:
    print("loop event ouput===")
    scanner.write(event['PayloadPart']['Bytes'])
    for line in scanner.readlines():
        resp = json.loads(line)
        print("line===")
        #print(resp.get("outputs")[0], end='')
        print(resp.get("outputs"), end='')
        

loop event ouput===
line===
{'delta': '', 'response': '', 'finished': False}loop event ouput===
line===
{'delta': '新加坡', 'response': '新加坡', 'finished': False}loop event ouput===
line===
{'delta': '是一个', 'response': '新加坡是一个', 'finished': False}loop event ouput===
line===
{'delta': '充满活力', 'response': '新加坡是一个充满活力', 'finished': False}loop event ouput===
line===
{'delta': '和', 'response': '新加坡是一个充满活力和', 'finished': False}loop event ouput===
line===
{'delta': '多元', 'response': '新加坡是一个充满活力和多元', 'finished': False}loop event ouput===
line===
{'delta': '文化', 'response': '新加坡是一个充满活力和多元文化', 'finished': False}loop event ouput===
line===
{'delta': '的城市', 'response': '新加坡是一个充满活力和多元文化的城市', 'finished': False}loop event ouput===
line===
{'delta': '国家', 'response': '新加坡是一个充满活力和多元文化的城市国家', 'finished': False}loop event ouput===
line===
{'delta': '，', 'response': '新加坡是一个充满活力和多元文化的城市国家，', 'finished': False}loop event ouput===
line===
{'delta': '有许多', 'response': '新加坡是一个充满活力和多元文化的城市国家，有许多', 'finished': False

JSONDecodeError: Unterminated string starting at: line 1 column 338 (char 337)

## Clean up the environment

In [ ]:
sess.delete_endpoint(endpoint_name)
sess.delete_endpoint_config(endpoint_name)
model.delete_model()